In [9]:
import random
import pandas as pd
from datetime import datetime, timedelta

# 1. КОНФИГУРАЦИЯ ГЕНЕРАТОРА

**Назначение**:  
Основные настройки для генерации тестовых CDR данных

**Формат номеров**:  
Все номера телефонов начинаются на `8` (формат `89XXXXXXXXX`)

**Коды операторов**:
- `98`, `99` - оператор "Ромашка"
- `91`, `92` - другие операторы
- `00`, `12` - невалидные коды (для негативных тестов)

**Параметры генерации**:
```python
NUM_VALID_RECORDS = 100    # Количество валидных записей
NUM_INVALID_RECORDS = 20   # Количество невалидных записей
MIN_CALL_DURATION = 10     # Минимальная длительность звонка (секунды)
MAX_CALL_DURATION = 3600   # Максимальная длительность (1 час = 3600 сек)

In [10]:
NUM_VALID_RECORDS = 100
NUM_INVALID_RECORDS = 20
ROMASHKA_OPERATORS = ['98', '99']  # Коды оператора Ромашка
OTHER_OPERATORS = ['91', '92']     # Коды других операторов
INVALID_OPERATORS = ['00', '12']   # Невалидные коды
MIN_CALL_DURATION = 10             # Минимальная длительность звонка (сек)
MAX_CALL_DURATION = 3600           # Максимальная длительность (1 час)

 # 2. ГЕНЕРАЦИЯ ВАЛИДНЫХ ДАННЫХ

**Назначение**:  
Функции для генерации корректных CDR записей, соответствующих спецификации

**Основные функции**:

### `generate_phone(operator)`
Генерирует валидный номер телефона формата `8XXXXXXXXXX`

**Параметры**:
- `operator` - код оператора (например '98' для Ромашки)

**Пример**:
```python
generate_phone('98')  # → '8987654321'

In [12]:
def generate_phone(operator):
    """Генерация номера телефона формата 8<код оператора>XXXXXXX"""
    return f"8{operator}{random.randint(1000000, 9999999):07d}"

def generate_valid_timestamps():
    """Создание корректных временных меток ISO 8601"""
    start = datetime.now() - timedelta(days=random.randint(0, 365))
    duration = timedelta(seconds=random.randint(MIN_CALL_DURATION, MAX_CALL_DURATION))
    end = start + duration
    return start.isoformat(), end.isoformat()

def generate_valid_record():
    """Создание одной валидной CDR записи"""
    call_type = random.choice(['01', '02'])  # 01 - исходящий, 02 - входящий

    # Для звонков Ромашки используем только их коды операторов
    if call_type == '01':
        from_op = random.choice(ROMASHKA_OPERATORS)
        to_op = random.choice(OTHER_OPERATORS + ROMASHKA_OPERATORS)
    else:
        from_op = random.choice(OTHER_OPERATORS + ROMASHKA_OPERATORS)
        to_op = random.choice(ROMASHKA_OPERATORS)

    start, end = generate_valid_timestamps()

    return [
        call_type,
        generate_phone(from_op),
        generate_phone(to_op),
        start,
        end
    ]

# 3. ГЕНЕРАЦИЯ НЕВАЛИДНЫХ ДАННЫХ

**Назначение**:  
Создание CDR записей с различными типами ошибок для тестирования валидации

**Основные функции**:

### `generate_invalid_phone()`
Генерирует номер телефона с ошибками

**Типы ошибок**:
1. Неверный код оператора (например, '8001234567')
2. Некорректная длина номера:
   - Слишком длинный (11+ цифр)
   - Слишком короткий (<10 цифр)
3. Пустое значение (None)

**Примеры вывода**:
```python
'8001234567'  # неверный оператор
'812345678901'  # слишком длинный
None  # пустое значение

In [13]:

def generate_invalid_phone():
    """Генерация невалидного номера телефона"""
    error_types = [
        lambda: f"8{random.choice(INVALID_OPERATORS)}{random.randint(1000000, 9999999):07d}",  # Неверный оператор
        lambda: str(random.randint(1000000000, 9999999999)),  # Слишком длинный номер
        lambda: "8" + str(random.randint(1000000, 9999999)),  # Слишком короткий номер
        lambda: None  # Пустое значение
    ]
    return random.choice(error_types)()

def generate_invalid_record():
    """Создание одной невалидной CDR записи"""
    error_type = random.choice([
        'invalid_operator',
        'invalid_number',
        'invalid_date',
        'empty_fields'
    ])

    call_type = random.choice(['01', '02'])

    if error_type == 'invalid_operator':
        return [
            call_type,
            generate_phone(random.choice(INVALID_OPERATORS)),
            generate_phone(random.choice(ROMASHKA_OPERATORS)),
            *generate_valid_timestamps()
        ]
    elif error_type == 'invalid_number':
        return [
            call_type,
            generate_invalid_phone(),
            generate_phone(random.choice(ROMASHKA_OPERATORS)),
            *generate_valid_timestamps()
        ]
    elif error_type == 'invalid_date':
        return [
            call_type,
            generate_phone(random.choice(ROMASHKA_OPERATORS)),
            generate_phone(random.choice(ROMASHKA_OPERATORS)),
            "2025-02-30T10:00:00",  # Несуществующая дата
            "2025-02-30T10:05:00"
        ]
    else:  # empty_fields
        return [
            call_type,
            None,  # Пропущенное поле
            generate_phone(random.choice(ROMASHKA_OPERATORS)),
            *generate_valid_timestamps()
        ]

# 4. СОЗДАНИЕ И СОХРАНЕНИЕ ФАЙЛОВ

**Назначение**:  
Формирование итоговых CSV-файлов с тестовыми данными

**Процесс генерации**:

1. **Создание наборов данных**:
```python
valid_data = [...]  # 100 валидных записей
invalid_data = [...]  # 20 невалидных записей

In [14]:
# Генерация данных
valid_data = [generate_valid_record() for _ in range(NUM_VALID_RECORDS)]
invalid_data = [generate_invalid_record() for _ in range(NUM_INVALID_RECORDS)]

# Создание DataFrame
columns = ['call_type', 'from_number', 'to_number', 'start_time', 'end_time']
valid_df = pd.DataFrame(valid_data, columns=columns)
invalid_df = pd.DataFrame(invalid_data, columns=columns)


In [19]:
# Примеры данных
print("=== Пример валидных записей ===")
print(valid_df.head().to_string(index=False))
print("\n=== Пример невалидных записей ===")
print(invalid_df.head().to_string(index=False))

=== Пример валидных записей ===
call_type from_number  to_number                 start_time                   end_time
       01  8991061788 8913703518 2025-03-05T19:05:05.282560 2025-03-05T19:43:40.282560
       02  8925572196 8987974975 2024-10-16T19:05:05.282642 2024-10-16T19:08:28.282642
       01  8999136767 8994318048 2025-02-07T19:05:05.282661 2025-02-07T19:05:39.282661
       01  8984390295 8981968329 2024-08-03T19:05:05.282680 2024-08-03T19:50:22.282680
       01  8999014696 8982822525 2024-12-18T19:05:05.282699 2024-12-18T19:59:20.282699

=== Пример невалидных записей ===
call_type from_number  to_number                 start_time                   end_time
       02  8128761478 8992126668 2024-12-28T19:05:05.284062 2024-12-28T19:24:09.284062
       02  8121245823 8984379517 2024-05-16T19:05:05.284083 2024-05-16T19:25:54.284083
       02  8122785511 8982625787 2025-04-15T19:05:05.284093 2025-04-15T19:33:00.284093
       01  8996778215 8986324132        2025-02-30T10:00:00    

In [20]:
# Сохранение в CSV
valid_df.to_csv('valid_cdr.csv', index=False, header=False)
invalid_df.to_csv('invalid_cdr.csv', index=False, header=False)

# Создание zip-архива для удобного скачивания
import zipfile
with zipfile.ZipFile('cdr_test_data.zip', 'w') as zipf:
    zipf.write('valid_cdr.csv')
    zipf.write('invalid_cdr.csv')

# Скачивание файлов
from google.colab import files

print("\nФайлы успешно созданы:")
print("- valid_cdr.csv (100 валидных записей)")
print("- invalid_cdr.csv (20 невалидных записей)")
print("- cdr_test_data.zip (оба файла в архиве)")

print("\nНачинается скачивание архива...")
files.download('cdr_test_data.zip')


Файлы успешно созданы:
- valid_cdr.csv (100 валидных записей)
- invalid_cdr.csv (20 невалидных записей)
- cdr_test_data.zip (оба файла в архиве)

Начинается скачивание архива...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>